In [1]:
from glob import glob
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import StratifiedShuffleSplit
import itertools
from collections import defaultdict

all_model_scores = defaultdict(list)

# For every data set in the data repository...
for dataset in glob('data/ecoli.csv.gz'):
    # Read the data set into memory
    input_data = pd.read_csv(dataset, compression='gzip')
    
    for dataset_repeat in range(1, 31):
        # Divide the data set into a training and testing sets, each time with a different RNG seed
        training_indices, testing_indices = next(iter(StratifiedShuffleSplit(input_data['class'].values,
                                                                             n_iter=1,
                                                                             train_size=0.75,
                                                                             test_size=0.25,
                                                                             random_state=dataset_repeat)))
        
        training_features = input_data.loc[training_indices].drop('class', axis=1).values
        training_classes = input_data.loc[training_indices, 'class'].values

        testing_features = input_data.loc[testing_indices].drop('class', axis=1).values
        testing_classes = input_data.loc[testing_indices, 'class'].values
        
        # These are parameters listed in the project document
        n_neighbors_list = [1, 5, 10, 50, 100]
        weights_list = ['uniform', 'distance']
        # For every DecisionTreeClassifier parameter combination...
        for (n_neighbors, weights) in itertools.product(n_neighbors_list, weights_list):
            # Create and fit the model on the training data
            clf = KNeighborsClassifier(n_neighbors = n_neighbors, weights = weights)
            clf.fit(training_features, training_classes)
            
            # Store the model's score with the key (name of data set, model, model parameters)
            all_model_scores[(dataset,
                              'KNeighborsClassifier',
                              (n_neighbors, weights))].append(clf.score(testing_features,
                                                                                      testing_classes))

In [3]:
import os
import gzip

if not os.path.isdir('benchmark_results'):
    os.mkdir('benchmark_results')

with gzip.open('benchmark_results/Kneighborslassifier_eoli-benchmarks.tsv.gz', 'wb') as out_file:
    header_text = '\t'.join(['dataset',
                              'model',
                              'parameters',
                              'testing_score']) + '\n'
    out_file.write(header_text.encode('UTF-8'))
    
    for (dataset, model, params) in all_model_scores:
        param_string = ''
        param_string += 'n_neighbors={},'.format(params[0])
        param_string += 'weights={},'.format(params[1])
        
        for testing_score in all_model_scores[(dataset, model, params)]:
            out_text = '\t'.join([dataset[5:-7],
                                  model,
                                  param_string,
                                  str(testing_score)]) + '\n'
            out_file.write(out_text.encode('UTF-8'))

In [4]:
import pandas as pd

pd.read_csv('benchmark_results/KNeighborsClassifier_eoli-benchmarks.tsv.gz', compression='gzip', sep='\t')

,dataset,model,parameters,testing_score
0,ecoli,KNeighborsClassifier,"n_neighbors=5,weights=distance,",0.890244
1,ecoli,KNeighborsClassifier,"n_neighbors=5,weights=distance,",0.841463
2,ecoli,KNeighborsClassifier,"n_neighbors=5,weights=distance,",0.926829
3,ecoli,KNeighborsClassifier,"n_neighbors=5,weights=distance,",0.853659
4,ecoli,KNeighborsClassifier,"n_neighbors=5,weights=distance,",0.792683
5,ecoli,KNeighborsClassifier,"n_neighbors=5,weights=distance,",0.865854
6,ecoli,KNeighborsClassifier,"n_neighbors=5,weights=distance,",0.914634
7,ecoli,KNeighborsClassifier,"n_neighbors=5,weights=distance,",0.853659
8,ecoli,KNeighborsClassifier,"n_neighbors=5,weights=distance,",0.865854
9,ecoli,KNeighborsClassifier,"n_neighbors=5,weights=distance,",0.853659
